In [3]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')  # Suppress warnings

In [51]:
df.dtypes

EST_TS            object
EST_DT            object
HR                 int64
MINUTE             int64
STORE             object
DASHER_ID        float64
D2R              float64
LATE_FLG           int64
DELIV_CLAT       float64
COMPLAINT          int64
WAS_REQUESTED      int64
WAS_MISSING        int64
WAS_SUBBED         int64
WAS_FOUND          int64
ITEM_CATEGORY     object
ITEM_NAME         object
SUB_CAT           object
SUB_ITEM          object
ITEM_PRICE       float64
dtype: object

In [75]:
df = pd.read_csv("../Data/prepped_data.csv")  # relative path

# Convert once and save
df['timestamp'] = pd.to_datetime(df['EST_TS'])

# Extract day of week name and number
df['DOW'] = df['timestamp'].dt.day_name()
# df['DOW_NBR'] = df['timestamp'].dt.weekday

# Create dummies
df = pd.get_dummies(df, columns=['STORE', 'DOW'])

df.head()


,EST_TS,EST_DT,HR,MINUTE,DASHER_ID,D2R,LATE_FLG,DELIV_CLAT,COMPLAINT,WAS_REQUESTED,...,STORE_Grocery1,STORE_Grocery2,STORE_Grocery3,DOW_Friday,DOW_Monday,DOW_Saturday,DOW_Sunday,DOW_Thursday,DOW_Tuesday,DOW_Wednesday
0,10/15/22 3:59,10/15/22,3,59,16990594.0,0.083333,0,0.366667,0,1,...,1,0,0,0,0,1,0,0,0,0
1,10/15/22 3:59,10/15/22,3,59,16990594.0,0.083333,0,0.366667,0,1,...,1,0,0,0,0,1,0,0,0,0
2,10/15/22 3:59,10/15/22,3,59,16990594.0,0.083333,0,0.366667,0,1,...,1,0,0,0,0,1,0,0,0,0
3,10/15/22 3:59,10/15/22,3,59,16990594.0,0.083333,0,0.366667,0,1,...,1,0,0,0,0,1,0,0,0,0
4,10/15/22 3:59,10/15/22,3,59,16990594.0,0.083333,0,0.366667,0,1,...,1,0,0,0,0,1,0,0,0,0


In [77]:
# step 1 correlation matrix for key variables
#### Lates, High correlation with how long for dasher to get to store
#### complaints, discrepency with store complaint, least from DM, most from grocery 1
#### Dasher not associated with complaints/lates/long accept times/long drives, is associated with specific stores
#### CERTAIN DOW associated with more/less lates indicating dow seasonality for lates
target_columns = ['LATE_FLG', 'COMPLAINT', 'DASHER_ID']

for col in target_columns:
    corr_matrix = df.corr()[[col]].drop(index=col).sort_values(by=col, ascending=False)
    print(f"\nCorrelation with '{col}':")
    print(corr_matrix)

# late_corr = df.corr()[['LATE_FLG']].drop(index='LATE_FLG').sort_values(by='LATE_FLG', ascending=False)


Correlation with 'LATE_FLG':
                 LATE_FLG
DELIV_CLAT       0.421570
D2R              0.079021
COMPLAINT        0.033381
STORE_DashMart1  0.032599
MINUTE           0.028414
HR               0.024348
DASHER_ID        0.023963
ITEM_PRICE       0.015290
DOW_Monday       0.015170
STORE_Grocery2   0.012166
DOW_Friday       0.008396
WAS_MISSING      0.003046
DOW_Saturday     0.002370
DOW_Thursday     0.001377
WAS_FOUND        0.000992
DOW_Sunday      -0.002097
WAS_SUBBED      -0.002570
DOW_Wednesday   -0.010747
DOW_Tuesday     -0.016737
STORE_Grocery3  -0.024801
STORE_Grocery1  -0.037476
WAS_REQUESTED         NaN

Correlation with 'COMPLAINT':
                 COMPLAINT
STORE_Grocery1    0.074553
DELIV_CLAT        0.067022
HR                0.035959
LATE_FLG          0.033381
WAS_SUBBED        0.029484
WAS_MISSING       0.027997
STORE_Grocery2    0.026691
DOW_Wednesday     0.012617
D2R               0.012190
DOW_Friday        0.008692
DOW_Thursday      0.007887
DOW_Sunday       